### Голощапова Алла
### 306 группа

# Задание
Для датасета https://archive.ics.uci.edu/dataset/336/chronic+kidney+disease
Обучить SVM, подобрать параметры, посчитать метрики качества классификации
Обучить Decision Tree Classifier, подобрать параметры, посчитать метрики качества классификации

In [1]:
import numpy as np
import pandas as pd

from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder
from sklearn.metrics import classification_report, accuracy_score
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split

from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV

In [2]:
data = pd.read_csv('https://archive.ics.uci.edu/static/public/336/data.csv')
data

,age,bp,sg,al,su,rbc,pc,pcc,ba,bgr,...,pcv,wbcc,rbcc,htn,dm,cad,appet,pe,ane,class
0,48.0,80.0,1.020,1.0,0.0,NaN,normal,notpresent,notpresent,121.0,...,44.0,7800.0,5.2,yes,yes,no,good,no,no,ckd
1,7.0,50.0,1.020,4.0,0.0,NaN,normal,notpresent,notpresent,NaN,...,38.0,6000.0,NaN,no,no,no,good,no,no,ckd
2,62.0,80.0,1.010,2.0,3.0,normal,normal,notpresent,notpresent,423.0,...,31.0,7500.0,NaN,no,yes,no,poor,no,yes,ckd
3,48.0,70.0,1.005,4.0,0.0,normal,abnormal,present,notpresent,117.0,...,32.0,6700.0,3.9,yes,no,no,poor,yes,yes,ckd
4,51.0,80.0,1.010,2.0,0.0,normal,normal,notpresent,notpresent,106.0,...,35.0,7300.0,4.6,no,no,no,good,no,no,ckd
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,55.0,80.0,1.020,0.0,0.0,normal,normal,notpresent,notpresent,140.0,...,47.0,6700.0,4.9,no,no,no,good,no,no,notckd
396,42.0,70.0,1.025,0.0,0.0,normal,normal,notpresent,notpresent,75.0,...,54.0,7800.0,6.2,no,no,no,good,no,no,notckd
397,12.0,80.0,1.020,0.0,0.0,normal,normal,notpresent,notpresent,100.0,...,49.0,6600.0,5.4,no,no,no,good,no,no,notckd
398,17.0,60.0,1.025,0.0,0.0,normal,normal,notpresent,notpresent,114.0,...,51.0,7200.0,5.9,no,no,no,good,no,no,notckd


In [3]:
data

,age,bp,sg,al,su,rbc,pc,pcc,ba,bgr,...,pcv,wbcc,rbcc,htn,dm,cad,appet,pe,ane,class
0,48.0,80.0,1.020,1.0,0.0,NaN,normal,notpresent,notpresent,121.0,...,44.0,7800.0,5.2,yes,yes,no,good,no,no,ckd
1,7.0,50.0,1.020,4.0,0.0,NaN,normal,notpresent,notpresent,NaN,...,38.0,6000.0,NaN,no,no,no,good,no,no,ckd
2,62.0,80.0,1.010,2.0,3.0,normal,normal,notpresent,notpresent,423.0,...,31.0,7500.0,NaN,no,yes,no,poor,no,yes,ckd
3,48.0,70.0,1.005,4.0,0.0,normal,abnormal,present,notpresent,117.0,...,32.0,6700.0,3.9,yes,no,no,poor,yes,yes,ckd
4,51.0,80.0,1.010,2.0,0.0,normal,normal,notpresent,notpresent,106.0,...,35.0,7300.0,4.6,no,no,no,good,no,no,ckd
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,55.0,80.0,1.020,0.0,0.0,normal,normal,notpresent,notpresent,140.0,...,47.0,6700.0,4.9,no,no,no,good,no,no,notckd
396,42.0,70.0,1.025,0.0,0.0,normal,normal,notpresent,notpresent,75.0,...,54.0,7800.0,6.2,no,no,no,good,no,no,notckd
397,12.0,80.0,1.020,0.0,0.0,normal,normal,notpresent,notpresent,100.0,...,49.0,6600.0,5.4,no,no,no,good,no,no,notckd
398,17.0,60.0,1.025,0.0,0.0,normal,normal,notpresent,notpresent,114.0,...,51.0,7200.0,5.9,no,no,no,good,no,no,notckd


В ходе кодировки категориальных данных, я заметила, что у таргета значиния 0, 1 и 2, что странно, тк были str: ckd и notckd!!!

In [4]:
cat_cols = [col for col in data.columns if data[col].dtype == 'object']
for col in cat_cols:
    print(f'{col} {data[col].unique()}')

rbc [nan 'normal' 'abnormal']
pc ['normal' 'abnormal' nan]
pcc ['notpresent' 'present' nan]
ba ['notpresent' 'present' nan]
htn ['yes' 'no' nan]
dm ['yes' 'no' '\tno' nan]
cad ['no' 'yes' nan]
appet ['good' 'poor' nan]
pe ['no' 'yes' nan]
ane ['no' 'yes' nan]
class ['ckd' 'ckd\t' 'notckd']


## Предобработка данных



**Текст, выделенный полужирным шрифтом**## Во первых, исправим "сломанные" данные.
***заменить ckd\t на ckd и \tno на no*** !!!

In [5]:
data = data.dropna()

In [6]:
#признаки

X = data.drop(['class'], axis=1)
X['dm'].replace({'\tno':'no', '\tyes':'yes', ' yes':'yes'}, inplace=True)
# X['cad'] = X['cad'].str.replace('\tno', 'no')
X

<ipython-input-6-3af3e6b4240b>:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  X['dm'].replace({'\tno':'no', '\tyes':'yes', ' yes':'yes'}, inplace=True)


,age,bp,sg,al,su,rbc,pc,pcc,ba,bgr,...,hemo,pcv,wbcc,rbcc,htn,dm,cad,appet,pe,ane
3,48.0,70.0,1.005,4.0,0.0,normal,abnormal,present,notpresent,117.0,...,11.2,32.0,6700.0,3.9,yes,no,no,poor,yes,yes
9,53.0,90.0,1.020,2.0,0.0,abnormal,abnormal,present,notpresent,70.0,...,9.5,29.0,12100.0,3.7,yes,yes,no,poor,no,yes
11,63.0,70.0,1.010,3.0,0.0,abnormal,abnormal,present,notpresent,380.0,...,10.8,32.0,4500.0,3.8,yes,yes,no,poor,yes,no
14,68.0,80.0,1.010,3.0,2.0,normal,abnormal,present,present,157.0,...,5.6,16.0,11000.0,2.6,yes,yes,yes,poor,yes,no
20,61.0,80.0,1.015,2.0,0.0,abnormal,abnormal,notpresent,notpresent,173.0,...,7.7,24.0,9200.0,3.2,yes,yes,yes,poor,yes,yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,55.0,80.0,1.020,0.0,0.0,normal,normal,notpresent,notpresent,140.0,...,15.7,47.0,6700.0,4.9,no,no,no,good,no,no
396,42.0,70.0,1.025,0.0,0.0,normal,normal,notpresent,notpresent,75.0,...,16.5,54.0,7800.0,6.2,no,no,no,good,no,no
397,12.0,80.0,1.020,0.0,0.0,normal,normal,notpresent,notpresent,100.0,...,15.8,49.0,6600.0,5.4,no,no,no,good,no,no
398,17.0,60.0,1.025,0.0,0.0,normal,normal,notpresent,notpresent,114.0,...,14.2,51.0,7200.0,5.9,no,no,no,good,no,no


In [7]:
cat_cols = [col for col in X.columns if X[col].dtype == 'object']
num_cols = [col for col in X.columns if X[col].dtype == 'float64']
for col in cat_cols:
    print(f'{col} {X[col].unique()}')

rbc ['normal' 'abnormal']
pc ['abnormal' 'normal']
pcc ['present' 'notpresent']
ba ['notpresent' 'present']
htn ['yes' 'no']
dm ['no' 'yes']
cad ['no' 'yes']
appet ['poor' 'good']
pe ['yes' 'no']
ane ['yes' 'no']


In [8]:
num_cols

['age',
 'bp',
 'sg',
 'al',
 'su',
 'bgr',
 'bu',
 'sc',
 'sod',
 'pot',
 'hemo',
 'pcv',
 'wbcc',
 'rbcc']

In [9]:
y = data['class'].map({'ckd':0, 'ckd\t':0, 'notckd':1})
y = pd.to_numeric(y, errors = 'coerce')
y

,class
3,0
9,0
11,0
14,0
20,0
...,...
395,1
396,1
397,1
398,1


In [10]:

# Кодирование категориальных признаков
text_columns = X.select_dtypes(include=['object']).columns
numeric_columns = X.select_dtypes(include=['number']).columns

encoder = OneHotEncoder(sparse_output=False, drop='first')
encoded_text = encoder.fit_transform(X[text_columns])

encoded_df = pd.DataFrame(encoded_text, index=X.index, columns=encoder.get_feature_names_out(text_columns))
final_df = pd.concat([X[numeric_columns], encoded_df], axis=1)
X = final_df

X

,age,bp,sg,al,su,bgr,bu,sc,sod,pot,...,rbc_normal,pc_normal,pcc_present,ba_present,htn_yes,dm_yes,cad_yes,appet_poor,pe_yes,ane_yes
3,48.0,70.0,1.005,4.0,0.0,117.0,56.0,3.8,111.0,2.5,...,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0
9,53.0,90.0,1.020,2.0,0.0,70.0,107.0,7.2,114.0,3.7,...,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0
11,63.0,70.0,1.010,3.0,0.0,380.0,60.0,2.7,131.0,4.2,...,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0
14,68.0,80.0,1.010,3.0,2.0,157.0,90.0,4.1,130.0,6.4,...,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0
20,61.0,80.0,1.015,2.0,0.0,173.0,148.0,3.9,135.0,5.2,...,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,55.0,80.0,1.020,0.0,0.0,140.0,49.0,0.5,150.0,4.9,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
396,42.0,70.0,1.025,0.0,0.0,75.0,31.0,1.2,141.0,3.5,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
397,12.0,80.0,1.020,0.0,0.0,100.0,26.0,0.6,137.0,4.4,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
398,17.0,60.0,1.025,0.0,0.0,114.0,50.0,1.0,135.0,4.9,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Из категориальных признаков в числовые

проверка типов данных, чтобы в дальнейшем не было проблем

In [11]:
X.dtypes

,0
age,float64
bp,float64
sg,float64
al,float64
su,float64
bgr,float64
bu,float64
sc,float64
sod,float64
pot,float64


### Замена NaN на медиану в столбце

In [12]:

# imputer= SimpleImputer(strategy='median')

# X = imputer.fit_transform(df_X)

# X

### Разбиение выборки на тестовую и тренировочную

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train.dtypes

,0
age,float64
bp,float64
sg,float64
al,float64
su,float64
bgr,float64
bu,float64
sc,float64
sod,float64
pot,float64


### Масштабирование

In [14]:
scaler = StandardScaler()
scaled_X_train = scaler.fit_transform(X_train)
scaled_X_test = scaler.transform(X_test)
scaled_X_train

array([[ 0.9376511 , -0.36791634,  0.0146474 , ..., -0.35355339,
        -0.38138504, -0.32444284],
       [ 0.34693091,  0.52357326,  0.0146474 , ..., -0.35355339,
        -0.38138504, -0.32444284],
       [ 0.67510879,  0.52357326,  0.9374334 , ..., -0.35355339,
        -0.38138504, -0.32444284],
       ...,
       [ 0.41256648, -0.36791634, -0.90813861, ..., -0.35355339,
        -0.38138504, -0.32444284],
       [ 0.47820206, -1.25940594,  0.0146474 , ..., -0.35355339,
        -0.38138504, -0.32444284],
       [-0.37506044, -0.36791634,  0.9374334 , ..., -0.35355339,
        -0.38138504, -0.32444284]])

## **Подбор параметров для метода опорных векторов**

In [15]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

# Подбор параметров SVM
param_grid = {'C': [0.1, 1, 10, 100, 1000], 'gamma': [1, 0.1, 0.01, 0.001, 0.0001], 'kernel': ['rbf']}
grid = GridSearchCV(SVC(), param_grid, refit=True, verbose=3)
grid.fit(X_train, y_train)

print("Лучшие параметры:", grid.best_params_)

Fitting 5 folds for each of 25 candidates, totalling 125 fits
[CV 1/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.731 total time=   0.0s
[CV 2/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.760 total time=   0.0s
[CV 3/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.720 total time=   0.0s
[CV 4/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.720 total time=   0.0s
[CV 5/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.720 total time=   0.0s
[CV 1/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.731 total time=   0.0s
[CV 2/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.760 total time=   0.0s
[CV 3/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.720 total time=   0.0s
[CV 4/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.720 total time=   0.0s
[CV 5/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.720 total time=   0.0s
[CV 1/5] END .....C=0.1, gamma=0.01, kernel=rbf;, score=0.731 total time=   0.0s
[CV 2/5] END .....C=0.1, gamma=0.01, kernel=rbf

In [16]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, precision_score, recall_score, f1_score

# Обучение SVM с лучшими параметрами
svm_model = grid.best_estimator_
svm_model.fit(scaled_X_train, y_train)

y_pred = svm_model.predict(scaled_X_test)

print("Метрики классификации:")
print(classification_report(y_test, y_pred))
print("Матрица ошибок:")
print(confusion_matrix(y_test, y_pred))
print("Accuracy:", accuracy_score(y_test, y_pred))

Метрики классификации:
              precision    recall  f1-score   support

           0       1.00      0.89      0.94         9
           1       0.96      1.00      0.98        23

    accuracy                           0.97        32
   macro avg       0.98      0.94      0.96        32
weighted avg       0.97      0.97      0.97        32

Матрица ошибок:
[[ 8  1]
 [ 0 23]]
Accuracy: 0.96875


In [17]:
# Метрики качества классификации
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')
report = classification_report(y_test, y_pred)

print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1 Score: {f1}')

Accuracy: 0.96875
Precision: 0.9700520833333334
Recall: 0.96875
F1 Score: 0.9681633291614518


## **Решающее дерево**

In [18]:
clf = DecisionTreeClassifier(random_state=42)
# clf.fit(scaled_X_train, y_train)

In [19]:
param_grid = {
    'max_depth': [10, 20, 30, 40, 50],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

grid_search = GridSearchCV(estimator=clf, param_grid=param_grid, scoring='accuracy', cv=5, n_jobs=-1)
grid_search.fit(scaled_X_train, y_train)
print("Лучшие параметры:", grid_search.best_params_)


svm_model = grid_search.best_estimator_
svm_model.fit(scaled_X_train, y_train)

y_pred = svm_model.predict(scaled_X_test)
best_clf = grid_search.best_estimator_

y_pred = best_clf.predict(scaled_X_test)

print("Метрики классификации:")
print(classification_report(y_test, y_pred))
print("Матрица ошибок:")
print(confusion_matrix(y_test, y_pred))
print("Accuracy:", accuracy_score(y_test, y_pred))

Лучшие параметры: {'max_depth': 10, 'min_samples_leaf': 1, 'min_samples_split': 2}
Метрики классификации:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         9
           1       1.00      1.00      1.00        23

    accuracy                           1.00        32
   macro avg       1.00      1.00      1.00        32
weighted avg       1.00      1.00      1.00        32

Матрица ошибок:
[[ 9  0]
 [ 0 23]]
Accuracy: 1.0


In [20]:

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')
report = classification_report(y_test, y_pred)

print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1 Score: {f1}')

Accuracy: 1.0
Precision: 1.0
Recall: 1.0
F1 Score: 1.0
